# Enhanced Embeddings with Top 5 Models

We implement separate embeddings with weights for the **`title`**, **`plot`** and **`genres`** fields using the 3 different sentence transformer models, time the embedding processes, and save each to separate files.

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Union
import time

c:\Users\danil\anaconda3\envs\dls\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
df = pd.read_csv('final_dataset.csv')
df['genres_str'] = df['genres'].apply(lambda x: ' '.join(x))

## Models Selection

3 models were chosen based on the [Sentence-Transformers leaderboard](https://www.sbert.net/docs/pretrained_models.html):

1. **all-mpnet-base-v2**: Highest general-purpose accuracy (12-layer, 768-hidden-dimension)
4. **all-MiniLM-L12-v2**: Balance of speed and performance (12-layer, 384-hidden-dimension)
5. **multi-qa-distilbert-cos-v1**: Fastest model with QA optimization, has been specifically trained for Semantic Search (6-layer, 768-hidden-dimension)  

In [3]:
WEIGHTS = {'title': 0.15, 'plot': 0.6, 'genres': 0.25}

MODELS = {
    'all-mpnet-base-v2',
    'all-MiniLM-L12-v2',
    'multi-qa-distilbert-cos-v1'
}


## Embedding Generation

We'll generate embeddings for each model, time the process, and save the results.

In [4]:
import time
import numpy as np
from sentence_transformers import SentenceTransformer
import os

os.makedirs('embeddings', exist_ok=True)

def generate_embeddings(model_name, texts):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(texts, convert_to_tensor=True)
    embeddings = [emb.cpu() for emb in embeddings]
    return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Dictionary to store timing results
timing_results = {}

for model_name in MODELS:
    start_time = time.time()
    
    # Generate all embeddings
    title_start = time.time()
    title_emb = generate_embeddings(model_name, df['title'].tolist())
    title_time = time.time() - title_start
    
    plot_start = time.time()
    plot_emb = generate_embeddings(model_name, df['plot'].tolist())
    plot_time = time.time() - plot_start
    
    genres_start = time.time()
    genres_emb = generate_embeddings(model_name, df['genres_str'].tolist())
    genres_time = time.time() - genres_start
    
    # Combine with stable weights
    combined_emb = (
        WEIGHTS['title'] * title_emb +
        WEIGHTS['plot'] * plot_emb + 
        WEIGHTS['genres'] * genres_emb
    )
    
    
    filename = f"embeddings/embeddings_{model_name.replace('-', '_')}.npy"
    np.save(filename, combined_emb)
    
    # Calculate total time
    total_time = time.time() - start_time
    
    # Store timing results
    timing_results[model_name] = {
        'title_embedding_time': title_time,
        'plot_embedding_time': plot_time,
        'genres_embedding_time': genres_time,
        'total_time': total_time
    }

In [5]:
with open('embedding_generation_times.txt', 'a') as f:
    for model_name, times in timing_results.items():
        f.write(f"Model: {model_name}\n")
        f.write(f"  Title embeddings: {times['title_embedding_time']:.2f} seconds\n")
        f.write(f"  Plot embeddings: {times['plot_embedding_time']:.2f} seconds\n")
        f.write(f"  Genres embeddings: {times['genres_embedding_time']:.2f} seconds\n")
        f.write(f"  Total time: {times['total_time']:.2f} seconds\n\n")

print("Embedding generation times saved to 'embedding_generation_times.txt'")

Embedding generation times saved to 'embedding_generation_times.txt'
